# Setup

## Imports

In [1]:
# Local
from fncs import *
from fncs_cluster_pos import *
from get_ecotones import get_ecotones

# General
import os
from tqdm.notebook import tqdm as log_progress
from datetime import datetime, timedelta
from configparser import ConfigParser

# Data analysis
import pandas as pd
import numpy as np

# # Data visualization
# import matplotlib.pyplot as plt
# import plotly.express as px
# import plotly.graph_objects as go
# from  plotly.subplots import make_subplots
# import matplotlib as mpl

# # Machine learning
# from sklearn.cluster import HDBSCAN, DBSCAN
# from st_dbscan import ST_DBSCAN
# from sklearn.model_selection import ParameterGrid
# from wdbscan import *
# from scipy.spatial.distance import pdist, squareform
# from sklearn.neighbors import KernelDensity

# Geospatial data
import geopandas as gpd
# from shapely.geometry import Point, LineString, LinearRing, MultiPoint, Polygon,MultiPolygon
# from scipy.spatial import distance_matrix
# from shapely import line_interpolate_point
import rasterio
# from rasterstats import zonal_stats
# from skimage import measure
# from shapelysmooth import  taubin_smooth
# from shapely import make_valid, distance

# # Other
# import networkx as nx
# import pickle


## Config

In [2]:
config = ConfigParser()
config.read(r'C:\Users\Tobi\Documents\BFNME Lokal\Buzzards2\config.ini')

['C:\\Users\\Tobi\\Documents\\BFNME Lokal\\Buzzards2\\config.ini']

# Load

## Files

In [3]:
tracks,ids = load_raw_files(config['Path']['tracks'])

Loading files:   0%|          | 0/65 [00:00<?, ?it/s]

In [4]:
gdfs_new = []
for gdf in tracks:
    gdf["dt_last_dp"] = gdf.index.get_level_values("datetime").diff()
    gdf["dt_next_dp"] = -gdf.index.get_level_values("datetime").diff(-1)
    gdf["dur"] = (gdf.dt_last_dp/2 + gdf.dt_next_dp/2).fillna(timedelta(hours=1))
    gdf["doy"] = gdf.index.get_level_values("datetime").dayofyear
    gdfs_new.append(gdf)
tracks = gdfs_new

In [5]:
tracks_new = []
sr_geoms_all = []

# kde for each sr
offset = 1000
min_sample_n = 50
cell_size = 75
bandwidth = 100
kde_percentile = 0.95
kde_reduction = True

for gdf in log_progress(tracks):
    track_new, sr_geoms = cluster_track(gdf, min_sample_n, offset, cell_size, bandwidth, kde_percentile, kde_reduction)
    sr_geoms = sr_geoms.geometry # besser in cluster track lösen
    tracks_new.append(track_new)
    sr_geoms_all.append(sr_geoms)
tracks = tracks_new

  0%|          | 0/64 [00:00<?, ?it/s]

SRs:   0%|          | 0/5 [00:00<?, ?it/s]

Contours:   0%|          | 0/1 [00:00<?, ?it/s]

Contours:   0%|          | 0/10 [00:00<?, ?it/s]

Contours:   0%|          | 0/8 [00:00<?, ?it/s]

...

Contours:   0%|          | 0/15 [00:00<?, ?it/s]

SRs:   0%|          | 0/3 [00:00<?, ?it/s]

Contours:   0%|          | 0/38 [00:00<?, ?it/s]

SRs:   0%|          | 0/2 [00:00<?, ?it/s]

Contours:   0%|          | 0/1 [00:00<?, ?it/s]

## geometrical features of srs

In [6]:
sr_geom_features_all = []
for sr_geoms in log_progress(sr_geoms_all, desc="Geometric properties"):
    sr_geom_features = pd.DataFrame({("area",""):sr_geoms.geometry.area})
    sr_geom_features_all.append(sr_geom_features)

Geometric properties:   0%|          | 0/64 [00:00<?, ?it/s]

## time features for srs

In [7]:
sr_params_all = []
for gdf in log_progress(tracks, desc="Time features"):
    sr_params = gdf.reset_index("datetime"
                             ).groupby("cluster"
                                       ).agg({'dur': 'sum', 'datetime': ['min', 'max']}
                                             )
    # sr_params.columns = [f"{c[0]}_{c[1]}" for c in sr_params.columns]
    sr_params_all.append(sr_params)

Time features:   0%|          | 0/64 [00:00<?, ?it/s]

## land cover features of srs

In [8]:
sr_zs_all = []

clc_crs = rasterio.open(config["CLC"]["path"]).crs
clc_legend = pd.read_csv(config["CLC"]["legend_path"],header=None)
clc_cls = clc_legend.iloc[:,5]
clc_values = [*range(1,45),48]
clc_key = {key: value for key, value in zip(clc_values, clc_cls)}

for sr_geoms in log_progress(sr_geoms_all, desc="Zonal stats"):
    sr_zs = zonal_stats_for_raster(sr_geoms,
                                config["CLC"]["path"],
                                clc_cls,
                                clc_crs,
                                clc_key)
    sr_zs.columns = pd.MultiIndex.from_product([sr_zs.columns, ["part"]])
    sr_zs_all.append(sr_zs)

Zonal stats:   0%|          | 0/64 [00:00<?, ?it/s]

In [12]:
import geopandas as gpd
import rasterio
import pandas as pd
from shapely import GeometryCollection, LineString, MultiLineString, Polygon
from shapelysmooth import taubin_smooth
from rasterio.features import shapes
from rasterio.mask import mask
from shapely.ops import linemerge



def get_ecotones(regions,exact=False):
    cls_land_cover = {10: "Tree cover", 20: "Shrubland", 30: "Grassland", 40: "Cropland", 50: "Built-up", 60: "Bare / sparse vegetation", 70: "Snow and Ice", 80: "Permanent water bodies", 90:"Herbaceous wetland", 100:"Moss and lichen"}
    land_cover_path = "C:/Users/Tobi/Documents/BFNME Lokal/Buzzard qgis/word land cover 2021 v2/ESA_WorldCover_10m_2021_v200_Map.tif"
    land_cover = rasterio.open(land_cover_path)
    regions = regions.to_crs("EPSG:4326")
    mask_array,mask_transform = mask(land_cover,regions, crop=True, filled=False)

    polygons = []
    categories = []
    origins = []
    for shape, value in shapes(mask_array,transform=mask_transform,connectivity=4):
        polygon = Polygon(shape["coordinates"][0], shape["coordinates"][1:])
        category = cls_land_cover[value]
        origin = regions.index[regions.intersects(polygon)][0] # statt der 0 vllt noch ne schönere Lösung finden
        polygons.append(polygon)
        categories.append(category)
        origins.append(origin)
    s = gpd.GeoDataFrame({"geometry":polygons,
                        "value":categories,
                        "origin":origins},
                        crs=land_cover.crs)
    borders = []
    for origin in s.origin.unique():
        s_sub = s[s.origin == origin]
        for i, poly1 in s_sub.iterrows():
            for j, poly2 in s_sub.iterrows():
                if i < j :
                    shared_boundary = poly1['geometry'].boundary.intersection(poly2['geometry'].boundary)

                    if not shared_boundary.is_empty:
                        if isinstance(shared_boundary, GeometryCollection):
                            shared_boundary = MultiLineString([geom for geom in shared_boundary.geoms if isinstance(geom, LineString)])
                        if isinstance(shared_boundary, MultiLineString):
                            shared_boundary = linemerge(shared_boundary)
                        if isinstance(shared_boundary, LineString):
                            shared_boundary = [shared_boundary]
                        elif isinstance(shared_boundary, MultiLineString):
                            shared_boundary = list(shared_boundary.geoms)
                        else:
                            continue
                        for line in shared_boundary:
                            if isinstance(line, LineString):
                                line = taubin_smooth(line)
                                borders.append({"geometry":line,poly1.value:True,poly2.value:True,"origin":origin})
    
    pd.set_option('future.no_silent_downcasting', True)
    borders = gpd.GeoDataFrame.from_dict(borders).fillna(False)
    pd.reset_option('future.no_silent_downcasting')

    land_cover_cls = s.value.unique()
    if exact:
        borders_new = []
        for i,group in borders.groupby("origin"):
            group = group.groupby(list(land_cover_cls),as_index=False)['geometry'].apply(lambda x: MultiLineString(x.tolist()))
            group.sort_index(axis=1, inplace=True)
            # index instead of dummies
            group.index = group.apply(lambda row: ' - '.join([col for col in group.columns.drop("geometry") if row[col]]), axis=1)
            group.drop(group.columns.drop("geometry"),axis=1,inplace=True)
            group["origin"] = i
            group["length"] = group.geometry.length
            group["len_percent"] = group["length"] / group["length"].sum()
            group["len_per_area"] = group["length"] / regions.loc[i].area
            borders_new.append(group)
        borders = pd.concat(borders_new)
        borders_wide = []
        for i,group in borders.groupby("origin"):
            df = group.drop(columns="origin").unstack().to_frame().T.swaplevel(0,1,axis=1).sort_index(axis=1)
            borders_wide.append(df)
        borders_wide = pd.concat(borders_wide,keys=borders.origin.unique()).droplevel(1)
        return borders_wide
    else:
        borders_new =[]
        for i,group in borders.groupby("origin"):
            data = []
            for land_cover in group.columns.drop(["geometry","origin"]):
                geom = MultiLineString(group.loc[group.loc[:,land_cover],"geometry"].to_list())
                data.append({"geometry":geom,"land_cover":land_cover})
            data = gpd.GeoDataFrame(data)
            data = data.set_index("land_cover")
            data["length"] = data.geometry.length
            data["len_percent"] = data["length"] / data["length"].sum()
            data["len_per_area"] = data["length"] / regions.loc[i].area
            data = data.unstack().to_frame().T.swaplevel(0,1,axis=1).sort_index(axis=1)
            # data["origin"] = i
            borders_new.append(data)
        borders_wide = pd.concat(borders_new,keys=borders.origin.unique()).droplevel(1)
        return borders_wide

In [14]:
sr_ecotones_all = []
for sr_geoms in log_progress(sr_geoms_all, desc="Ecotones"):
    sr_ecotones = get_ecotones(sr_geoms)
    sr_ecotones_all.append(sr_ecotones)

Ecotones:   0%|          | 0/64 [00:00<?, ?it/s]

In [16]:
srs = []
for sr_params, sr_zs, sr_ecotones, sr_geom_features, sr_geoms in zip(sr_params_all, sr_zs_all, sr_ecotones_all, sr_geom_features_all, sr_geoms_all):
    srs_temp = pd.concat([sr_params, sr_zs, sr_ecotones, sr_geom_features], axis=1, keys=['params', 'zs', 'ecotones', 'geom']).sort_index()
    srs_temp = gpd.GeoDataFrame(srs_temp, geometry=sr_geoms)
    srs.append(srs_temp)